In [15]:
!pip install flask_sqlalchemy


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: C:\Users\bayram\AppData\Local\Programs\Python\Python311\python.exe -m pip install --upgrade pip


## Demo

In [64]:
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from marshmallow import Schema, fields, validate, ValidationError
from werkzeug.security import generate_password_hash, check_password_hash
import threading

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db = SQLAlchemy(app)

class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True, nullable=False)
    password = db.Column(db.String(120), nullable=False)

class UserSchema(Schema):
    id = fields.Int(dump_only=True)
    username = fields.Str(required=True, validate=validate.Length(min=1, max=80))
    password = fields.Str(required=True, validate=validate.Length(min=6))

user_schema = UserSchema()
users_schema = UserSchema(many=True)

with app.app_context():
    db.create_all()

@app.route('/')
def home():
    return "Welcome to the Security Testing Demo!"

@app.route('/users', methods=['GET'])
def get_users():
    try:
        users = User.query.all()
        result =  users_schema.dump(users)
        return jsonify(result)
    except Exception as e:
        return jsonify({"message": "An error occurred while fetching users", "error": str(e)}), 500


@app.route('/user/<int:id>', methods=['GET'])
def get_user(id):
    user = User.query.get(id)
    if user:
      result = user_schema.dump(user)
      return jsonify(result)
    return jsonify({"message": "User not found"}), 404

@app.route('/user', methods=['POST'])
def add_user():
    data = request.get_json()
    errors = user_schema.validate(data)
    if errors:
        return jsonify(errors), 400

    hashed_password = generate_password_hash(data['password'])
    new_user = User(username=data['username'], password=hashed_password)
    db.session.add(new_user)
    db.session.commit()
    return jsonify({"message": "User added successfully"}), 201

@app.route('/user/<int:id>', methods=['PUT'])
def update_user(id):
    data = request.get_json()
    errors = user_schema.validate(data)
    if errors:
        return jsonify(errors), 400

    user = User.query.get(id)
    if user:
        user.username = data['username']
        user.password = generate_password_hash(data['password'])
        db.session.commit()
        return jsonify({"message": "User updated successfully"})
    return jsonify({"message": "User not found"}), 404

@app.route('/user/<int:id>', methods=['DELETE'])
def delete_user(id):
    user = User.query.get(id)
    if user:
        db.session.delete(user)
        db.session.commit()
        return jsonify({"message": "User deleted successfully"})
    return jsonify({"message": "User not found"}), 404

threading.Thread(target=app.run, kwargs={'host':'0.0.0.0','port':5000}).start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [65]:
!curl -X GET http://127.0.0.1:5000/users

[2025-01-07 13:56:51,856] ERROR in app: Exception on /users [GET]
Traceback (most recent call last):
  File "C:\Users\bayram\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1463, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bayram\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 872, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bayram\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 870, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bayram\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 855, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

<!doctype html>
<html lang=en>
<title>500 Internal Server Error</title>
<h1>Internal Server Error</h1>
<p>The server encountered an internal error and was unable to complete your request. Either the server is overloaded or there is an error in the application.</p>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   265  100   265    0     0  24014      0 --:--:-- --:--:-- --:--:-- 24090


In [53]:
!curl -X GET http://127.0.0.1:5000/user/1

C:\Users\bayram\AppData\Local\Temp\ipykernel_4240\3906710096.py:29: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  user = User.query.get(id)
127.0.0.1 - - [07/Jan/2025 13:51:03] "GET /user/1 HTTP/1.1" 404 -


{"message":"User not found"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0    29    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    29  100    29    0     0   1038      0 --:--:-- --:--:-- --:--:--  1035


In [58]:
!curl -X POST http://localhost:5000/user -H "Content-Type: application/json" -d '{"username":"testuser","password":"securepassword"}'


127.0.0.1 - - [07/Jan/2025 13:55:07] "POST /user HTTP/1.1" 400 -


<!doctype html>
<html lang=en>
<title>400 Bad Request</title>
<h1>Bad Request</h1>
<p>The browser (or proxy) sent a request that this server could not understand.</p>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   212  100   167  100    45    691    186 --:--:-- --:--:-- --:--:--   879


In [55]:
!curl -X PUT http://localhost:5000/user/1 -H "Content-Type: application/json" -d '{"username":"testuser","password":"newsecurepassword"}'

127.0.0.1 - - [07/Jan/2025 13:51:06] "PUT /user/1 HTTP/1.1" 400 -


<!doctype html>
<html lang=en>
<title>400 Bad Request</title>
<h1>Bad Request</h1>
<p>The browser (or proxy) sent a request that this server could not understand.</p>


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100   215  100   167  100    48    701    201 --:--:-- --:--:-- --:--:--   907


In [56]:
!curl -X DELETE http://localhost:5000/user/1

C:\Users\bayram\AppData\Local\Temp\ipykernel_4240\3906710096.py:55: LegacyAPIWarning: The Query.get() method is considered legacy as of the 1.x series of SQLAlchemy and becomes a legacy construct in 2.0. The method is now available as Session.get() (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  user = User.query.get(id)
127.0.0.1 - - [07/Jan/2025 13:51:07] "DELETE /user/1 HTTP/1.1" 404 -


{"message":"User not found"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    29  100    29    0     0    134      0 --:--:-- --:--:-- --:--:--   134


In [57]:
import unittest
import requests
import json

BASE_URL = "http://127.0.0.1:5000"

class FlaskAppTestCase(unittest.TestCase):

    def test_sql_injection(self):
        payload = {"username": "testuser'; DROP TABLE users; --", "password": "testpass"}
        response = requests.post(f"{BASE_URL}/user", json=payload)
        self.assertNotEqual(response.status_code, 201, "SQL Injection vulnerability detected")

    def test_xss(self):
        payload = {"username": "<script>alert('XSS');</script>", "password": "testpass"}
        response = requests.post(f"{BASE_URL}/user", json=payload)
        self.assertNotIn("<script>", response.text, "XSS vulnerability detected")

    def test_insecure_password_storage(self):
        payload = {"username": "testuser", "password": "testpass"}
        response = requests.post(f"{BASE_URL}/user", json=payload)
        self.assertEqual(response.status_code, 201)
        user_id = response.json()["id"]
        response = requests.get(f"{BASE_URL}/user/{user_id}")
        self.assertNotIn("testpass", response.json()["password"], "Insecure password storage detected")

    def test_authentication(self):
        response = requests.get(f"{BASE_URL}/users")
        self.assertEqual(response.status_code, 401, "No authentication detected")

    def test_authorization(self):
        payload = {"username": "admin", "password": "adminpass"}
        response = requests.post(f"{BASE_URL}/user", json=payload)
        user_id = response.json()["id"]
        payload = {"username": "normaluser", "password": "userpass"}
        response = requests.post(f"{BASE_URL}/user", json=payload)
        response = requests.put(f"{BASE_URL}/user/{user_id}", json={"username": "hacked", "password": "hackedpass"})
        self.assertEqual(response.status_code, 403, "No authorization detected")

    def test_insecure_direct_object_references(self):
        payload = {"username": "testuser1", "password": "testpass"}
        response = requests.post(f"{BASE_URL}/user", json=payload)
        user_id = response.json()["id"]
        response = requests.get(f"{BASE_URL}/user/{user_id}")
        self.assertNotEqual(response.status_code, 200, "Insecure direct object reference detected")

    def test_data_exposure(self):
        response = requests.get(f"{BASE_URL}/users")
        self.assertNotIn("password", response.json()[0], "Sensitive data exposure detected")

unittest.main()

E
ERROR: C:\Users\bayram\AppData\Roaming\jupyter\runtime\kernel-4d585098-27ee-4d1c-8507-20e137179edb (unittest.loader._FailedTest.C:\Users\bayram\AppData\Roaming\jupyter\runtime\kernel-4d585098-27ee-4d1c-8507-20e137179edb)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\bayram\AppData\Roaming\jupyter\runtime\kernel-4d585098-27ee-4d1c-8507-20e137179edb'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

C:\Users\bayram\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
